In [1]:
r"""
Graphormer MHA alignment"""

'\nGraphormer MHA alignment'

In [2]:
# select device
import os 
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import torch
import mindspore as ms
import numpy as np

import transformers.models.graphormer.modeling_graphormer as ptm
import transformers.models.graphormer.configuration_graphormer as ptc

import mindnlp.transformers.models.graphormer.modeling_graphormer as msm
import mindnlp.transformers.models.graphormer.configuration_graphormer as msc

/home/hubo/Software/miniconda3/envs/mindspore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[WARNING] ME(35288:140340131664256,MainProcess):2023-11-15-18:12:58.435.815 [mindspore/run_check/_check_version.py:102] MindSpore version 2.2.0 and cuda version 11.4.148 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.
2023-11-15 18:13:00,092 - modelscope - INFO - PyTorch version 1.12.0+cu116 Found.
2023-11-15 18:13:00,093 - modelscope - INFO - Loading ast index from /home/hubo/.cache/modelscope/ast_indexer
2023-11-15 18:13:00,123 - modelscope - INFO - Loading done! Current index file version is 1.9.4, with md5 d9a86a06b3e111926fdc559cc3e2c156 and a total number

In [4]:
# init config
ms_config = msc.GraphormerConfig()
pt_config = ptc.GraphormerConfig()

In [5]:
# init model
ms_model = msm.GraphormerMultiheadAttention(ms_config)
pt_model = ptm.GraphormerMultiheadAttention(pt_config)

In [6]:
def ms_param_name_to_pt(param_name: str):
    if 'embedding_table' in param_name:
        param_name = param_name.replace('embedding_table', 'weight')
    elif 'layer_norm'in param_name:
        param_name = param_name.replace('layer_norm.gamma', 'layer_norm.weight')
        param_name = param_name.replace('layer_norm.beta', 'layer_norm.bias')
    return param_name

In [7]:
# print ms_model parameters' name
ms_keys = ms_model.parameters_dict().keys()
ms_keys = set([ms_param_name_to_pt(k) for k in ms_keys])

pt_params = dict(pt_model.named_parameters())
pt_keys = set(set(pt_params.keys()))
diff1 = sorted(ms_keys - pt_keys)
diff2 = sorted(pt_keys - ms_keys)
if len(diff1) or len(diff2):
    print(diff1)
    print(diff2)
else:
    print('Parameter names matched')

Parameter names matched


In [8]:
# load parameters
for key, param in ms_model.parameters_and_names():
    key = ms_param_name_to_pt(key)
    param.set_data(ms.Tensor(pt_params.get(key).detach().numpy()))

In [9]:
# set eval mode
ms_model.set_train(False)
pt_model.eval()

GraphormerMultiheadAttention(
  (attention_dropout_module): Dropout(p=0.1, inplace=False)
  (k_proj): Linear(in_features=768, out_features=768, bias=True)
  (v_proj): Linear(in_features=768, out_features=768, bias=True)
  (q_proj): Linear(in_features=768, out_features=768, bias=True)
  (out_proj): Linear(in_features=768, out_features=768, bias=True)
)

In [28]:
def ids_tensor_np(shape, vocab_size):
    print(vocab_size)
    return np.random.randint(0, vocab_size-1, shape)

def floats_tensor_np(shape, scale=1.0):
    return np.random.uniform(low=0, high=scale, size=shape)

In [47]:
def np_inputs_to_ms(inputs):
    inputs_ms = dict()
    for key, value in inputs.items():
        if key in ['attn_bias', 'query', 'key', 'value']:
            inputs_ms[key] = ms.tensor(value, dtype=ms.float32)
        elif key == 'key_padding_mask':
            inputs_ms[key] = ms.tensor(value, dtype=ms.dtype.bool_)
        else:
            inputs_ms[key] = ms.tensor(value, dtype=ms.int64)
    return inputs_ms

def np_inputs_to_pt(inputs):
    inputs_pt = dict()
    for key, value in inputs.items():
        if key in ['attn_bias', 'query', 'key', 'value']:
            inputs_pt[key] = torch.tensor(value, dtype=torch.float32)
        elif key == 'key_padding_mask':
            inputs_pt[key] = torch.tensor(value, dtype=torch.bool)
        else:
            inputs_pt[key] = torch.tensor(value, dtype=torch.int64)
    return inputs_pt
        

In [48]:
# prepare data
batch_size=10
graph_size=20

query = floats_tensor_np([graph_size + 1, batch_size, ms_config.embedding_dim])

inputs = dict(
    query = query,
    key = query,
    value = query,
    attn_bias = ids_tensor_np([batch_size * ms_config.num_attention_heads, graph_size + 1, graph_size + 1], ms_config.num_atoms * 2 + 1),
    key_padding_mask = np.full((batch_size, graph_size + 1), False))
additional_inputs = dict(need_weights = False, attn_mask = None)

inputs_ms = np_inputs_to_ms(inputs)
inputs_pt = np_inputs_to_pt(inputs)
inputs_ms.update(additional_inputs)
inputs_pt.update(additional_inputs)


9217


In [49]:
# output
ms_out = ms_model(**inputs_ms)
pt_out = pt_model(**inputs_pt)

In [53]:
# Check that the outputs have same shape
assert ms_out[0].shape == pt_out[0].shape

In [56]:
def judge(o1, o2, loss = 1e-3, prefix = '-'):
    prefix += '-'
    if (isinstance(o1, tuple)):
        for i in range(len(o1)):
            judge(o1[i], o2[i], loss=loss, prefix=prefix)
    elif (isinstance(o1, ms.Tensor)):
        np1 = o1.asnumpy()
        np2 = o2.detach().numpy()
        print(f"{prefix}{np.allclose(np1, np2, loss, loss)}")
        print(np.abs((np1 - np2)).max())
    else:
        print(f"{type(o1)}-{type(o2)}:{o1==o2}")


In [57]:
# Check that the outputs difference are within tolerable range
loss = 1e-5

judge(ms_out[0], pt_out[0], loss)

--True
4.7683716e-07
